## Feature Interactivity and Engineering

In [25]:
import pandas as pd
import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
#from geopy.distance import geodesic
import time
from sklearn.preprocessing import LabelEncoder
import geopy.distance as d
import numpy as np

In [2]:
train = pd.read_csv('./data/trainclean.csv')
test = pd.read_csv('./data/testclean.csv')

/Users/samlundberg/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (27,28,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### There is missing data in the dataset.
Many missing values are entered a M, - and T in the data set.  This code sets them all to -1 do distinquish from other data.

In [3]:
# replace some missing values and T with -1
train = train.replace('M', -1)
train = train.replace('-', -1)
train = train.replace('T', -1)
train = train.replace(' T', -1)
train = train.replace('  T', -1)

In [4]:
# replace some missing values and T with -1
test = test.replace('M', -1)
test = test.replace('-', -1)
test = test.replace('T', -1)
test = test.replace(' T', -1)
test = test.replace('  T', -1)

In [5]:
test.columns

Index(['index', 'id', 'date', 'address', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'year', 'month', 'day', 'tmax_x', 'tmin_x', 'tavg_x', 'depart_x',
       'dewpoint_x', 'wetbulb_x', 'heat_x', 'cool_x', 'sunrise_x', 'sunset_x',
       'codesum_x', 'depth_x', 'water1_x', 'snowfall_x', 'preciptotal_x',
       'stnpressure_x', 'sealevel_x', 'resultspeed_x', 'resultdir_x',
       'avgspeed_x', 'tmax_y', 'tmin_y', 'tavg_y', 'depart_y', 'dewpoint_y',
       'wetbulb_y', 'heat_y', 'cool_y', 'sunrise_y', 'sunset_y', 'codesum_y',
       'depth_y', 'water1_y', 'snowfall_y', 'preciptotal_y', 'stnpressure_y',
       'sealevel_y', 'resultspeed_y', 'resultdir_y', 'avgspeed_y',
       'species_culex_pipiens', 'species_culex_pipiens/restuans',
       'species_culex_restuans', 'species_culex_salinarius',
       'species_culex_tarsalis', 'species_culex_territans',
       'species_unspecified_culex', 'distance', 'distance.1'],
      dty

In [14]:
train['wetbulb_x'].value_counts()

69    1254
61     909
71     775
70     703
64     610
62     479
65     463
66     462
68     402
67     399
58     355
59     344
63     342
74     315
49     309
56     291
73     258
72     256
57     254
75     244
76     184
47     181
54     136
60     125
-1      93
50      93
55      93
52      77
51      60
53      26
46      14
Name: wetbulb_x, dtype: int64

In [21]:
train.dtypes

address                            object
addressaccuracy                     int64
addressnumberandstreet             object
avgspeed_x                        float64
avgspeed_y                        float64
block                               int64
codesum_x                          object
codesum_y                          object
cool_x                              int64
cool_y                              int64
date                               object
day                               float64
depart_x                            int64
depart_y                            int64
depth_x                             int64
depth_y                             int64
dewpoint_x                          int64
dewpoint_y                          int64
distance                          float64
heat_x                              int64
heat_y                              int64
index                             float64
latitude                          float64
longitude                         

In [20]:
train['block'] * train['wetbulb_x']

0        6565656565656565656565656565656565656565656565...
1        6565656565656565656565656565656565656565656565...
2        6565656565656565656565656565656565656565656565...
3        6565656565656565656565656565656565656565656565...
4        6565656565656565656565656565656565656565656565...
5                                               6565656565
6                                 656565656565656565656565
7                                                       65
8                                                       65
9                                                       65
10                                      656565656565656565
11                                    65656565656565656565
12                                    65656565656565656565
13                                    65656565656565656565
14                                    65656565656565656565
15                                                      65
16                                            6565656565

In [6]:
train.columns

Index(['address', 'addressaccuracy', 'addressnumberandstreet', 'avgspeed_x',
       'avgspeed_y', 'block', 'codesum_x', 'codesum_y', 'cool_x', 'cool_y',
       'date', 'day', 'depart_x', 'depart_y', 'depth_x', 'depth_y',
       'dewpoint_x', 'dewpoint_y', 'distance', 'heat_x', 'heat_y', 'index',
       'latitude', 'longitude', 'month', 'nummosquitos', 'preciptotal_x',
       'preciptotal_y', 'resultdir_x', 'resultdir_y', 'resultspeed_x',
       'resultspeed_y', 'sealevel_x', 'sealevel_y', 'snowfall_x', 'snowfall_y',
       'species_culex_pipiens', 'species_culex_pipiens/restuans',
       'species_culex_restuans', 'species_culex_salinarius',
       'species_culex_tarsalis', 'species_culex_territans', 'stnpressure_x',
       'stnpressure_y', 'street', 'sunrise_x', 'sunrise_y', 'sunset_x',
       'sunset_y', 'tavg_x', 'tavg_y', 'tmax_x', 'tmax_y', 'tmin_x', 'tmin_y',
       'trap', 'water1_x', 'water1_y', 'wetbulb_x', 'wetbulb_y', 'wnvpresent',
       'year'],
      dtype='object')

In [7]:
#train.drop(columns = ['codesum_x', 'codesum_y'], inplace=True)

**Trap, Block, and Street**

In [26]:
traps = train[['trap',
               'latitude',
               'longitude',
               'date',
               'wnvpresent',
               'nummosquitos']].groupby(['trap',
                                         'latitude',
                                         'longitude',
                                         'date', 
                                         'wnvpresent']).agg({'nummosquitos':np.sum}).reset_index()


In [36]:
traps

,trap,latitude,longitude,date,wnvpresent
0,0,41.953705,-87.733974,2007-06-26,0.0
1,0,41.953705,-87.733974,2007-07-11,0.0
2,0,41.953705,-87.733974,2007-07-18,0.0
3,0,41.953705,-87.733974,2007-08-01,0.0
4,0,41.953705,-87.733974,2007-08-07,0.0
5,0,41.953705,-87.733974,2007-08-15,0.0
6,0,41.953705,-87.733974,2007-08-21,0.0
7,0,41.953705,-87.733974,2007-08-24,0.0
8,0,41.953705,-87.733974,2007-09-12,0.0
9,0,41.953705,-87.733974,2007-10-04,0.0


In [30]:
traps.drop('nummosquitos',axis=1,inplace=True)

In [31]:
traps.shape

(4924, 5)

In [32]:
traps.drop_duplicates(inplace=True)

In [33]:
traps.shape

(4924, 5)

In [34]:
traps.head()

,trap,latitude,longitude,date,wnvpresent
0,0,41.953705,-87.733974,2007-06-26,0.0
1,0,41.953705,-87.733974,2007-07-11,0.0
2,0,41.953705,-87.733974,2007-07-18,0.0
3,0,41.953705,-87.733974,2007-08-01,0.0
4,0,41.953705,-87.733974,2007-08-07,0.0


In [37]:
train

,address,addressaccuracy,addressnumberandstreet,avgspeed_x,avgspeed_y,block,codesum_x,codesum_y,cool_x,cool_y,...,tmax_y,tmin_x,tmin_y,trap,water1_x,water1_y,wetbulb_x,wetbulb_y,wnvpresent,year
0,"4100 North Oak Park Avenue, Chicago, IL 60634,...",9,"4100 N OAK PARK AVE, Chicago, IL",6.5,7.4,26,BR HZ,BR HZ,9,12,...,88,60,65,1,-1,-1,65,66,0.0,2007.0
1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",9,"4100 N OAK PARK AVE, Chicago, IL",6.5,7.4,26,BR HZ,BR HZ,9,12,...,88,60,65,1,-1,-1,65,66,0.0,2007.0
2,"6200 North Mandell Avenue, Chicago, IL 60646, USA",9,"6200 N MANDELL AVE, Chicago, IL",6.5,7.4,42,BR HZ,BR HZ,9,12,...,88,60,65,6,-1,-1,65,66,0.0,2007.0
3,"7900 West Foster Avenue, Chicago, IL 60656, USA",8,"7900 W FOSTER AVE, Chicago, IL",6.5,7.4,55,BR HZ,BR HZ,9,12,...,88,60,65,13,-1,-1,65,66,0.0,2007.0
4,"7900 West Foster Avenue, Chicago, IL 60656, USA",8,"7900 W FOSTER AVE, Chicago, IL",6.5,7.4,55,BR HZ,BR HZ,9,12,...,88,60,65,13,-1,-1,65,66,0.0,2007.0
5,"1500 West Webster Avenue, Chicago, IL 60614, USA",8,"1500 W WEBSTER AVE, Chicago, IL",6.5,7.4,5,BR HZ,BR HZ,9,12,...,88,60,65,32,-1,-1,65,66,0.0,2007.0
6,"2500 West Grand Avenue, Chicago, IL 60654, USA",8,"2500 W GRAND AVE, Chicago, IL",6.5,7.4,12,BR HZ,BR HZ,9,12,...,88,60,65,33,-1,-1,65,66,0.0,2007.0
7,"1100 Roosevelt Road, Chicago, IL 60608, USA",8,"1100 W ROOSEVELT, Chicago, IL",6.5,7.4,1,BR HZ,BR HZ,9,12,...,88,60,65,35,-1,-1,65,66,0.0,2007.0
8,"1100 Roosevelt Road, Chicago, IL 60608, USA",8,"1100 W ROOSEVELT, Chicago, IL",6.5,7.4,1,BR HZ,BR HZ,9,12,...,88,60,65,35,-1,-1,65,66,0.0,2007.0
9,"1100 West Chicago Avenue, Chicago, IL 60642, USA",8,"1100 W CHICAGO, Chicago, IL",6.5,7.4,1,BR HZ,BR HZ,9,12,...,88,60,65,36,-1,-1,65,66,0.0,2007.0


In [8]:
lbl = LabelEncoder()
lbl.fit(list(train['trap'].values) + list(test['trap'].values))
train['trap'] = lbl.transform(train['trap'].values)
test['trap'] = lbl.transform(test['trap'].values)

In [9]:
lbl.fit(list(train['block'].values) + list(test['block'].values))
train['block'] = lbl.transform(train['block'].values)
test['block'] = lbl.transform(test['block'].values)

In [10]:
lbl.fit(list(train['street'].values) + list(test['street'].values))
train['street'] = lbl.transform(train['street'].values)
test['street'] = lbl.transform(test['street'].values)

In [11]:
#This is just a shortcut for now. If we rerun the original EDA doc, this row will kick an error out.
test['distance'] = test['distance.1']

In [12]:
test.drop(columns = 'distance.1', inplace=True)

**Distance and Species**

State or reiterate information about vertical transmission.

In [13]:
train['dist_species_culex_pipiens'] = train['species_culex_pipiens'] * train['distance']
test['dist_species_culex_pipiens'] = test['species_culex_pipiens'] * test['distance']

In [14]:
train['dist_species_culex_pipiens/restuans'] = train['species_culex_pipiens/restuans'] * train['distance']
test['dist_species_culex_pipiens/restuans'] = test['species_culex_pipiens/restuans'] * test['distance']

In [15]:
train['dist_species_culex_restuans'] = train['species_culex_restuans'] * train['distance']
test['dist_species_culex_restuans'] = test['species_culex_restuans'] * test['distance']

In [16]:
train['dist_species_culex_salinarius'] = train['species_culex_salinarius'] * train['distance']
test['dist_species_culex_salinarius'] = test['species_culex_salinarius'] * test['distance']

In [17]:
train['dist_species_culex_tarsalis'] = train['species_culex_tarsalis'] * train['distance']
test['dist_species_culex_tarsalis'] = test['species_culex_tarsalis'] * test['distance']

In [18]:
train['dist_species_culex_territans'] = train['species_culex_territans'] * train['distance']
test['dist_species_culex_territans'] = test['species_culex_territans'] * test['distance']

In [19]:
test['species_unspecified_culex_dist'] = test['species_unspecified_culex'] * test['distance']

**Temperature and Precipitation**

In [20]:
#Creating interaction terms below between temperature and precipitation (termed "Wetbulb"). Water seems like it would be similarly indicative of disease movement to precipitation, but most of the water values are missing from the initial dataset, so we won't use this as part of an interaction term.

In [21]:
#train['temp_and_precip_x'] = train['tmax_x'] * train['wetbulb_x'] 
#test['temp_and_precip_x'] = test['tmax_x'] * test['wetbulb_x'] 

In [22]:
#train['trap_and_precip_y'] = train['trap'] * train['wetbulb_y'] 
#test['trap_and_precip_x'] = test['trap'] * test['wetbulb_x']

In [23]:
#train['block_and_precip_x'] = train['block'] * train['wetbulb_x'] 
#test['block_and_precip_x'] = test['block'] * test['wetbulb_x'] 

**Sealevel Binary Variable Creation by Standard Deviation**

In [24]:
#We called .describe() on our sealevel variables with the idea that mosquitoes gravitate around water and water will pool in areas with low sealevel. We then engineered a term called "low_sealevel" that consists only of the lowest standard deviation of sealevel in the grouping.

In [25]:
train['sealevel_x'].describe()

count    10506.000000
mean        29.966956
std          0.116038
min         29.600000
25%         29.890000
50%         29.980000
75%         30.050000
max         30.330000
Name: sealevel_x, dtype: float64

In [26]:
test['sealevel_x'].describe() 

count    116293.000000
mean         29.985536
std           0.122456
min          29.710000
25%          29.890000
50%          29.990000
75%          30.070000
max          30.330000
Name: sealevel_x, dtype: float64

In [27]:
for sealevel in train['sealevel_x']:
    if sealevel <= 29.89000:
        train['low_sealevel_x'] = 1
    else:
        train['low_sealevel_x'] = 0

In [28]:
for sealevel in test['sealevel_x']:
    if sealevel <= 29.89000:
        test['low_sealevel_x'] = 1
    else:
        test['low_sealevel_x'] = 0

In [29]:
train['sealevel_y'].describe()

count    10506.000000
mean        29.954502
std          0.117755
min         29.590000
25%         29.870000
50%         29.960000
75%         30.040000
max         30.330000
Name: sealevel_y, dtype: float64

In [30]:
test['sealevel_y'].describe()

count    116293.000000
mean         29.970243
std           0.123908
min          29.690000
25%          29.870000
50%          29.980000
75%          30.050000
max          30.310000
Name: sealevel_y, dtype: float64

In [31]:
for sealevel in train['sealevel_y']:
    if sealevel <= 29.87000:
        train['low_sealevel_y'] = 1
    else:
        train['low_sealevel_y'] = 0

In [32]:
for sealevel in test['sealevel_y']:
    if sealevel <= 29.87000:
        test['low_sealevel_y'] = 1
    else:
        test['low_sealevel_y'] = 0

#### Month and Distance Interaction

In visualizing the data in Tableau, we can see that the West Nile cases start to be more prolific in the mid summer with fewer cases in the early and late summer so though that distance might interact with month.

In [33]:
train['month_and_dist'] = train['month'] * train['distance']
test['month_and_dist'] = test['month'] * test['distance']

**Sealevel and Month Interaction**

In [34]:
train['month_and_sealevel_x'] = train['month'] * train['low_sealevel_x']
test['month_and_sealevel_x'] = test['month'] * test['low_sealevel_x']

In [35]:
train['month_and_sealevel_y'] = train['month'] * train['low_sealevel_y']
test['month_and_sealevel_y'] = test['month'] * test['low_sealevel_y']

**Month, Distance, and Temperature**

In [36]:
train['month_dist_temp_x'] = train['month'] * train['distance'] * train['tavg_x']
test['month_dist_temp_x'] = test['month'] * test['distance'] * test['tavg_x']

In [37]:
train['month_dist_temp_y'] = train['month'] * train['distance'] * train['tavg_y']
test['month_dist_temp_y'] = test['month'] * test['distance'] * test['tavg_y']

In [38]:
train['month_dist_maxtemp_x'] = train['month'] * train['distance'] * train['tmax_x']
test['month_dist_maxtemp_x'] = test['month'] * test['distance'] * test['tmax_x']

In [39]:
train['month_dist_maxtemp_y'] = train['month'] * train['distance'] * train['tmax_y']
test['month_dist_maxtemp_y'] = test['month'] * test['distance'] * test['tmax_y']

**Month, Distance, and Sealevel**

In [40]:
train['month_dist_sealevel_x'] = train['month'] * train['distance'] * train['low_sealevel_x']
test['month_dist_sealevel_x'] = test['month'] * test['distance'] * test['low_sealevel_x']

In [41]:
train['month_dist_sealevel_y'] = train['month'] * train['distance'] * train['low_sealevel_y']
test['month_dist_sealevel_y'] = test['month'] * test['distance'] * test['low_sealevel_y']

In [42]:
train.columns

Index(['address', 'addressaccuracy', 'addressnumberandstreet', 'avgspeed_x',
       'avgspeed_y', 'block', 'codesum_x', 'codesum_y', 'cool_x', 'cool_y',
       'date', 'day', 'depart_x', 'depart_y', 'depth_x', 'depth_y',
       'dewpoint_x', 'dewpoint_y', 'distance', 'heat_x', 'heat_y', 'index',
       'latitude', 'longitude', 'month', 'nummosquitos', 'preciptotal_x',
       'preciptotal_y', 'resultdir_x', 'resultdir_y', 'resultspeed_x',
       'resultspeed_y', 'sealevel_x', 'sealevel_y', 'snowfall_x', 'snowfall_y',
       'species_culex_pipiens', 'species_culex_pipiens/restuans',
       'species_culex_restuans', 'species_culex_salinarius',
       'species_culex_tarsalis', 'species_culex_territans', 'stnpressure_x',
       'stnpressure_y', 'street', 'sunrise_x', 'sunrise_y', 'sunset_x',
       'sunset_y', 'tavg_x', 'tavg_y', 'tmax_x', 'tmax_y', 'tmin_x', 'tmin_y',
       'trap', 'water1_x', 'water1_y', 'wetbulb_x', 'wetbulb_y', 'wnvpresent',
       'year', 'dist_species_culex_pipiens',

In [43]:
test.columns

Index(['index', 'id', 'date', 'address', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'year', 'month', 'day', 'tmax_x', 'tmin_x', 'tavg_x', 'depart_x',
       'dewpoint_x', 'wetbulb_x', 'heat_x', 'cool_x', 'sunrise_x', 'sunset_x',
       'codesum_x', 'depth_x', 'water1_x', 'snowfall_x', 'preciptotal_x',
       'stnpressure_x', 'sealevel_x', 'resultspeed_x', 'resultdir_x',
       'avgspeed_x', 'tmax_y', 'tmin_y', 'tavg_y', 'depart_y', 'dewpoint_y',
       'wetbulb_y', 'heat_y', 'cool_y', 'sunrise_y', 'sunset_y', 'codesum_y',
       'depth_y', 'water1_y', 'snowfall_y', 'preciptotal_y', 'stnpressure_y',
       'sealevel_y', 'resultspeed_y', 'resultdir_y', 'avgspeed_y',
       'species_culex_pipiens', 'species_culex_pipiens/restuans',
       'species_culex_restuans', 'species_culex_salinarius',
       'species_culex_tarsalis', 'species_culex_territans',
       'species_unspecified_culex', 'distance', 'dist_species_culex_pipi

In [57]:
train['latitude'].head()

0    41.954690
1    41.954690
2    41.994991
3    41.974089
4    41.974089
Name: latitude, dtype: float64

In [58]:
train['longitude'].head()

0   -87.800991
1   -87.800991
2   -87.769279
3   -87.824812
4   -87.824812
Name: longitude, dtype: float64

In [62]:
def add_dist(both):
    subset=both[['latitude',"longitude"]]
    tuples=[tuple(x) for x in subset.values]
    both['point']=tuples
    
    def dist_ohare(point):
        return d.vicenity(point,ohare).miles
    
    ohare=(41.9742, -87.9073)
    both['d_airport']=both['point'].apply(dist_ohare)
    
    def dist_port(point):
        return d.vincentry(point, port).miles
    
    port=(41.673408, -87.599862)
    both['d_port']=both['point'].apply(dist_ohare)
    
    both.loc[both['d_airport'] < 3, 'd_airport_1'] =1
    both['d_airport_1']=0
             
    both.loc[both['d_port']<3, 'd_port_1'] = 1
    both['d_port_1']=0
            
    return both

**Saving the Next CSV for Analysis Notebook**

In [273]:
train.to_csv('./data/trainw.csv')
test.to_csv('./data/testw.csv')

In [ ]:
train.columns

In [ ]:
test.columns